In [16]:
train = False

# Introduction

- Introduction includes a brief description of your and what problem your
technique would be applied to
- Introduction includes a description of the dataset that you used
- Introduction includes and the specific inputs and outputs of the problem
- Introduction clearly states the two techniques used (see section below)
- Introduction clearly states the experimental question explored in this
project (see experiment section below)

# Emojify 
Is a Hand-Drawn Image to Emoji classification ML system that uses Convolutional 
Neural Networks. 

We used the drawn images from Google's "Quick, Draw!" dataset and labeled the output text with emoji's from https://emojipedia.org/.

Our system takes in a drawn image and outputs an emoji.

The two techniques we used were MobileNetv2 and 

Our experimental question is

# Downloading data

In [13]:
import os
# curl csv file from google drive
!curl "https://raw.githubusercontent.com/victorhuangwq/Emojify/main/data/text-to-emoji.csv" -o "dataset/text-to-emoji.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6889  100  6889    0     0  20020      0 --:--:-- --:--:-- --:--:-- 20381
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1207  100  1207    0     0   4945      0 --:--:-- --:--:-- --:--:--  4987- --:--:-- --:--:--     0


## Installing necessary packages

In [4]:
%pip install tensorflow-macos

  Using cached tensorflow_macos-2.12.0-cp38-cp38-macosx_12_0_arm64.whl (200.8 MB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached flatbuffers-23.3.3-py2.py3-none-any.whl (26 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached h5py-3.8.0-cp38-cp38-macosx_11_0_arm64.whl (2.6 MB)
  Using cached jax-0.4.8.tar.gz (1.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached libclang-16.0.0-py2.py3-none-macosx_11_0_arm64.whl (24.3 MB)
  Using cached numpy-1.23.5-cp38-cp38-macosx_11_0_arm64.whl (13.3 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached protobuf-4.22.3-cp37-abi3-macosx_10_9_universal2.whl (397 kB)
  Using cached termcolor-2.3.0-py3-none-any.whl (6.9 kB)
  Using cached wrapt-1.14.1-cp38-cp38-macosx_11_

In [9]:
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install tensorflow_datasets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 11.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 15.5 MB/s eta 0:00:00a 0:00:01
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.6/120.6 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 15.0 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21494 sha256=79ec1a881373c687a3ab1db3e9ddb255450b78afe01f42970bb7b95e4f0143c5
  S

In [10]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

/Users/victorhuang/code/Emojify/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Importing Data

In [12]:
# import quickdraw dataset
quickdraw_train = tfds.load('quickdraw_bitmap', split='train', shuffle_files=True)

2023-05-04 11:08:40.478004: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dl Completed...:   0%|          | 0/345 [00:24<?, ? url/s]


KeyboardInterrupt: 

# Load the models

In [15]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(200, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
model.fit(...)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(...)

0 input_4
1 conv2d_282
2 batch_normalization_282
3 activation_282
4 conv2d_283
5 batch_normalization_283
6 activation_283
7 conv2d_284
8 batch_normalization_284
9 activation_284
10 max_pooling2d_12
11 conv2d_285
12 batch_normalization_285
13 activation_285
14 conv2d_286
15 batch_normalization_286
16 activation_286
17 max_pooling2d_13
18 conv2d_290
19 batch_normalization_290
20 activation_290
21 conv2d_288
22 conv2d_291
23 batch_normalization_288
24 batch_normalization_291
25 activation_288
26 activation_291
27 average_pooling2d_27
28 conv2d_287
29 conv2d_289
30 conv2d_292
31 conv2d_293
32 batch_normalization_287
33 batch_normalization_289
34 batch_normalization_292
35 batch_normalization_293
36 activation_287
37 activation_289
38 activation_292
39 activation_293
40 mixed0
41 conv2d_297
42 batch_normalization_297
43 activation_297
44 conv2d_295
45 conv2d_298
46 batch_normalization_295
47 batch_normalization_298
48 activation_295
49 activation_298
50 average_pooling2d_28
51 conv2d_294
52